In [1]:
import os
import sys
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.append(parent_dir)

In [ ]:
%cd ..

In [ ]:
from src.prep_vid_v2 import get_images_tensor
import pyarrow.parquet as pq
from pathlib import Path
from tqdm import tqdm
from src.model import load_model
import torch
import pandas as pd

In [4]:
data_path = [Path('data/datasets/eval')]
models = ['adaface']

In [ ]:
for path in data_path:
    for model_name in models:
        dataset = path.parts[-1]
        vids = list(path.rglob('*.mp4'))
        data = {'video_id': [], 'person_id': [], 'embedding': [], 'frame_id': []}
        model = load_model(model_name)
        for video in tqdm(vids, desc=f'Model: {model_name}, Dataset: {dataset}'):
            video_id = video.stem
            person_id = video.parts[-2]
            images = get_images_tensor(video)
            if images is None or images.shape == torch.Size([0]):
                data['video_id'].append(video_id)
                data['person_id'].append(person_id)
                data['embedding'].append(None)
                data['frame_id'].append(None)
                # print(f'No faces detected in {video}')
                continue
            embeddings = model(images.float().cuda())
            if isinstance(embeddings, torch.Tensor):
                embeddings = embeddings.detach().numpy()
            for i, embedding in enumerate(embeddings):
                embedding = embedding.squeeze()
                data['video_id'].append(video_id)
                data['person_id'].append(person_id)
                data['embedding'].append(embedding)
                data['frame_id'].append(int(i))
        df = pd.DataFrame(data)
        df.to_parquet(f'data/fusion_win/test.parquet')